In [ ]:
from PIL import Image
import numpy as np
import os
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow import keras
import shutil

from skimage import io
# but no such module exists
#Augment Data but differently

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import f1_score, precision_score, recall_score


image_folder = "C:/Users/compsci6651/Desktop/Filtered_Images"
labels_file_path = 'C:/Users/compsci6651/PycharmProjects/SeniorSem/HAM10000_metadata.txt'
labels_dict = {}  # Assuming you have loaded this dictionary before calling the function
with open(labels_file_path, 'r', encoding='utf-8') as file:
    next(file)  # Skip the header row
    for line in file:
        parts = line.strip().split(',')
        image_id = parts[1]
        label = parts[2]
        if label in ['mel', 'nv']:
            labels_dict[image_id] = label

label_to_int = {'nv': 0, 'mel': 1}
image_paths = [os.path.join(image_folder, filename) for filename in os.listdir(image_folder) if filename.endswith('.jpg')]
image_labels = [label_to_int[labels_dict.get(filename.replace('.jpg', ''), "unknown")] for filename in os.listdir(image_folder) if filename.endswith('.jpg')]

augmented_folder = 'C:/Users/compsci6651/Desktop/augmented_images'
augmented_file_path = 'C:/Users/compsci6651/PycharmProjects/SeniorSem/Augmented'
os.makedirs(augmented_folder, exist_ok=True)

def flip_images_manually(image_folder, augmented_folder, labels_dict, label_to_int):
    # Iterate over all images and check their labels
    for image_filename in os.listdir(image_folder):
        if image_filename.lower().endswith('.jpg'):
            label = labels_dict.get(image_filename.replace('.jpg', ''), "unknown")
            if label == 'mel':  # Only augment images with 'mel' label
                image_path = os.path.join(image_folder, image_filename)
                with Image.open(image_path) as image:
                    # Horizontal flip
                    flipped_image_horizontal = image.transpose(Image.FLIP_LEFT_RIGHT)
                    flipped_image_horizontal.save(os.path.join(augmented_folder, f'horizontal_{image_filename}'))

                    # Vertical flip
                    flipped_image_vertical = image.transpose(Image.FLIP_TOP_BOTTOM)
                    flipped_image_vertical.save(os.path.join(augmented_folder, f'vertical_{image_filename}'))

                    # Flip both
                    flipped_image_both = flipped_image_vertical.transpose(Image.FLIP_LEFT_RIGHT)
                    flipped_image_both.save(os.path.join(augmented_folder, f'both_{image_filename}'))

# Call the function to process images
flip_images_manually(image_folder, augmented_folder, labels_dict, label_to_int)


with open(augmented_file_path, 'a') as label_file:
    for filename in os.listdir(augmented_folder):
        if filename.lower().endswith('.jpg') and (filename.startswith('horizontal_') or filename.startswith('vertical_') or filename.startswith('both_')):
            full_path = os.path.join(augmented_folder, filename)
            label_file.write(f'{full_path},mel\n')

datagen = ImageDataGenerator(
    rotation_range=20,       # Rotation range in degrees
    width_shift_range=0.2,   # Fraction of total width for horizontal shift
    height_shift_range=0.2,  # Fraction of total height for vertical shift
    zoom_range=0.2,          # Zoom range
    horizontal_flip=True,    # Enable horizontal flipping
    fill_mode='reflect'      # Use 'reflect' mode for filling gaps after a shift
)

dataset = []

my_images = os.listdir(image_folder)
for i, image_name in enumerate(my_images):
    if (image_name.split('.')[1] == 'jpg'):
        if (image_labels[i] == 1):
            image = io.imread(image_folder + '/' + image_name)
            image = Image.fromarray(image, 'RGB')
            dataset.append(np.array(image))


x = np.array(dataset)
print(len(x))

i = 0

for batch in datagen.flow(x, batch_size=1, save_to_dir=augmented_folder, save_prefix='aug_mel_', save_format='jpg'):
    i += 1
    if i >= 2253:  # Stop after 5592 images are generated
        break



with open(augmented_file_path, 'a') as label_file:
    for filename in os.listdir(augmented_folder):
        if filename.lower().endswith('.jpg') and filename.startswith('aug_mel_'):
            full_path = os.path.join(augmented_folder, filename)
            label_file.write(f'{full_path},mel\n')
